In [1]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import *

In [2]:
cancer_data = pd.read_csv('Cancer_Data.csv')
#cancer_data = cancer_data.drop(columns = ['Unnamed: 0'])
cancer_data = cancer_data.rename(columns = {"Unnamed: 0" : "ID"})
cancer_data = cancer_data.iloc[:,:13]
cancer_data.head(10)

,ID,diagnosis,children,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,0,M,Yes,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871
1,1,M,No,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667
2,2,M,Yes,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999
3,3,M,No,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744
4,4,M,Yes,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883
5,5,M,Yes,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613
6,6,M,No,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742
7,7,M,No,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451
8,8,M,Yes,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389
9,9,M,No,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243


In [45]:
df = cancer_data
df = df.round(decimals = 3)
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in dataframe.columns])),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

tabs_styles = {
    'height': '35px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

app.layout = html.Div([ 
        dcc.Tab(label='Data Overview', style=tab_style, selected_style=tab_selected_style,children=[
            html.Div([
                html.H1(children='Wisconsin Breast Cancer Dataset')
            ]),
            html.Div([
                html.Label('Diagnosis', style = {'fontSize': 20, 'font-weight': 'bold'}),
                dcc.Dropdown(
                    id = 'Diagnosis-selection',
                    options=[
                        {'label': 'Malignant', 'value': 'M'},
                        {'label': 'Benign', 'value': 'B'}
                    ],
                    value=['M', 'B'], 
                    multi=True),
                html.Br(),
                
                html.Label('Children', style = {'fontSize': 20, 'font-weight': 'bold'}),
                dcc.RadioItems(
                    id = 'Children-selection',
                    options=[
                        {'label': 'Yes', 'value': 'Y'},
                        {'label': 'No', 'value': 'N'},
                        {'label': 'Both', 'value': 'Both'}
                    ],
                    value='Both', 
                    labelStyle={'fontSize':16, 'text-indent': '20px', 'display': 'inline-block'}),
                html.Br(),
                
                html.Label('Radius Mean', style = {'fontSize': 20, 'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='radius-range-slider',
                    marks={i: '{}'.format(10 ** i) for i in range(5,30)},
                    max=30,
                    value=[5, 20],
                    dots=False,
                    step=0.5,
                    updatemode='drag'), 
                html.Div(id='output-container-range-slider-non-linear'),
                html.Br(),
                
                html.Label('Texture Mean', style = {'fontSize': 20, 'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='texture-range-slider',
                    marks={i: '{}'.format(10 ** i) for i in range(5,30)},
                    max=30,
                    value=[5, 20],
                    dots=False,
                    step=0.5,
                    updatemode='drag'),
                html.Div(id='output-container-texture-slider-non-linear'),
                html.Br(),
                
                 html.Div([
                    generate_table(df)
                ])
                
                
            ])       
        ]),
        
        
        dcc.Tab(label='Tab two', style=tab_style, selected_style=tab_selected_style,children=[
        ]),
        dcc.Tab(label='Tab three', style=tab_style, selected_style=tab_selected_style, children=[
        ]),
    ],style=tabs_styles)

def transform_value(value):
    return 10 ** value

@app.callback(
    Output('output-container-range-slider-non-linear'),
    [Input('radius-range-slider', 'value')],
    Output('output-container-texture-slider-non-linear'),
    [Input('texture-range-slider', 'value')])



def update_output(value):
    transformed_value = [transform_value(v) for v in value]
    return 'Radius Value: {}'.format(
        str(value),
        transformed_value[0],
        transformed_value[1]
    )


if __name__ == '__main__':
    app.run_server(debug=False)

TypeError: __init__() missing 1 required positional argument: 'component_property'